## Yahoo Finance

In [2]:
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import chromedriver_binary
import string
import datetime
from forex_python.converter import CurrencyRates
pd.options.display.float_format = '{:.3f}'.format

### Functions for screener results and getting data

In [2]:
def get_rate(from_currency, to_currency):
    curr_rates = CurrencyRates()
    return curr_rates.get_rate(from_currency.upper(), to_currency.upper())

def convert(value, rate):
    if value:
        return value * rate
    else:
        return ''

In [3]:
def set_dictionary(companies_info, ticker_info, symbol, curr_rate):
    companies_info[symbol] = {
        'symbol'           : ticker_info['symbol'] if 'symbol' in ticker_info else '',
        'shortName'        : ticker_info['shortName'] if 'shortName' in ticker_info else '',
        'country'          : ticker_info['country'] if 'country' in ticker_info else '',
        'averageVolume'    : ticker_info['averageVolume'] if 'averageVolume' in ticker_info else '',
        'currentPrice'     : ticker_info['currentPrice'] if 'currentPrice' in ticker_info else '',
        'currency'         : ticker_info['currency'] if 'currency' in ticker_info else '',
        'sector'           : ticker_info['sector'] if 'sector' in ticker_info else '',
        'industry'         : ticker_info['industry'] if 'industry' in ticker_info else '',
        'profitMargins'    : ticker_info['profitMargins'] if 'profitMargins' in ticker_info else '',
        'operatingMargins' : ticker_info['operatingMargins'] if 'operatingMargins' in ticker_info else '',
        'operatingCashflow': convert(ticker_info['operatingCashflow'],curr_rate) if 'operatingCashflow' in ticker_info else '',
        'returnOnAssets'   : ticker_info['returnOnAssets'] if 'returnOnAssets' in ticker_info else '',
        'returnOnEquity'   : ticker_info['returnOnEquity'] if 'returnOnEquity' in ticker_info else '',
        'trailingEps'      : convert(ticker_info['trailingEps'],curr_rate) if 'trailingEps' in ticker_info else '',
        'totalRevenue'     : convert(ticker_info['totalRevenue'],curr_rate) if 'totalRevenue' in ticker_info else '',
        'revenuePerShare'  : convert(ticker_info['revenuePerShare'],curr_rate) if 'revenuePerShare' in ticker_info else '',
        'revenueGrowth'    : ticker_info['revenueGrowth'] if 'revenueGrowth' in ticker_info else '',
        'grossProfits'     : convert(ticker_info['grossProfits'],curr_rate) if 'grossProfits' in ticker_info else '',
        'netIncomeToCommon': convert(ticker_info['netIncomeToCommon'],curr_rate) if 'netIncomeToCommon' in ticker_info else '',
        'operatingMargins' : ticker_info['operatingMargins'] if 'operatingMargins' in ticker_info else '',
        'totalCash'        : convert(ticker_info['totalCash'],curr_rate) if 'totalCash' in ticker_info else '',
        'totalCashPerShare': convert(ticker_info['totalCashPerShare'],curr_rate) if 'totalCashPerShare' in ticker_info else '',
        'totalDebt'        : convert(ticker_info['totalDebt'],curr_rate) if 'totalDebt' in ticker_info else '',
        'bookValue'        : ticker_info['bookValue'] if 'bookValue' in ticker_info else '',
        'marketCap'        : convert(ticker_info['marketCap'],curr_rate) if 'marketCap' in ticker_info else '',
        'enterpriseValue'  : convert(ticker_info['enterpriseValue'],curr_rate) if 'enterpriseValue' in ticker_info else '',
        'trailingPE'       : ticker_info['trailingPE'] if 'trailingPE' in ticker_info else '',
        'earningsQuarterlyGrowth': ticker_info['earningsQuarterlyGrowth'] if 'earningsQuarterlyGrowth' in ticker_info else '',
        'forwardPE'              : ticker_info['forwardPE'] if 'forwardPE' in ticker_info else '',
        'pegRatio'               : ticker_info['pegRatio'] if 'pegRatio' in ticker_info else '',
        'priceToSalesTrailing12Months': ticker_info['priceToSalesTrailing12Months'] if 'priceToSalesTrailing12Months' in ticker_info else '',
        'priceToBook'         : ticker_info['priceToBook'] if 'priceToBook' in ticker_info else '',
        'enterpriseToRevenue' : ticker_info['enterpriseToRevenue'] if 'enterpriseToRevenue' in ticker_info else '',
        'beta'                : ticker_info['beta'] if 'beta' in ticker_info else '',
        'fiftyTwoWeekHigh'    : convert(ticker_info['fiftyTwoWeekHigh'],curr_rate) if 'fiftyTwoWeekHigh' in ticker_info else '',
        'fiftyTwoWeekLow'     : convert(ticker_info['fiftyTwoWeekLow'],curr_rate) if 'fiftyTwoWeekLow' in ticker_info else '',
        'fiftyDayAverage'     : convert(ticker_info['fiftyDayAverage'],curr_rate) if 'fiftyDayAverage' in ticker_info else '',
        'twoHundredDayAverage': convert(ticker_info['twoHundredDayAverage'],curr_rate) if 'twoHundredDayAverage' in ticker_info else '',
        'trailingAnnualDividendYield' : ticker_info['trailingAnnualDividendYield'] if 'trailingAnnualDividendYield' in ticker_info else '',
        'fiveYearAvgDividendYield'    : ticker_info['fiveYearAvgDividendYield'] if 'fiveYearAvgDividendYield' in ticker_info else ''
    }
    
    return companies_info

In [4]:
"""
function for scraping screener results which takes two parameters url and currency,
so that we can reuse it for different countries screener 
"""
def get_companies(url):
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.execute_script('return document.body.innerHTML;')
    soup = BeautifulSoup(html,'html.parser')
    results = soup.find('table').find_all('tr')[1:]
    companies_results = []
    for tr in results:
        td = tr.find('td')
        companies_results.append(td.find('a').text)
    return companies_results


"""
Function for getting company financial informations from scraping results.
It takes parameter currency which is involved in a conditional. This conditional excludes stocks
with different currency from that of country/continent we're scraping.
"""
def get_companies_info(companies_results, currency, curr_rate, country):
    companies_info = dict() 
    for i in range(0, len(companies_results)):
        ct =  yf.Ticker(companies_results[i])
        print(ct.info['symbol'], end=' ')
        companies_info = set_dictionary(companies_info, ct.info, ct.info['symbol'],  curr_rate)
    return companies_info


def set_dataframe(companies_info):
    return pd.DataFrame(companies_info)

def save_to_csv(dataframe, filename):
    dataframe.to_csv(filename, sep=',', encoding='utf-8-sig', index=False)

In [5]:
# filter
def filter_info(given_currency, given_country, country_info):
    info = []
    info = [item for item in country_info if item['country'] == given_country or item['country'] == '']
    info1 = [item for item in info if item['currency'] == given_currency]
    return info1

## Turkey

In [115]:
turkey_url = 'https://finance.yahoo.com/screener/unsaved/46fa9c6a-b6eb-420a-8d51-9cbe4f700ddb?dependentField=sector&dependentValues=Financial%20Services'
turkey_results = get_companies(turkey_url)

In [117]:
turkey_results = ['ISBTR.IS',
 'QNBFB.IS',
 'GARAN.IS',
 'KLNMA.IS',
 'AKBNK.IS',
 'ISCTR.IS',
 'YKBNK.IS',
 'SAHOL.IS',
 'VAKBN.IS',
 'HALKB.IS',
 'ICBCT.IS',
 'TSKB.IS',
 'ALBRK.IS',
 'SKBNK.IS']

In [118]:
c = CurrencyRates()
turkey_rate = c.get_rate('TRY', 'USD')
turkey_rate

0.11626185564975375

In [119]:
turkey_companies_info = get_companies_info(turkey_results, 'TRY', turkey_rate, 'Turkey')

ISBTR.IS QNBFB.IS GARAN.IS KLNMA.IS AKBNK.IS ISCTR.IS YKBNK.IS SAHOL.IS VAKBN.IS HALKB.IS ICBCT.IS TSKB.IS ALBRK.IS SKBNK.IS 

In [120]:
turkey_companies_info2 = filter_info('TRY', 'Turkey', turkey_companies_info.values())
len(turkey_companies_info), len(turkey_companies_info2)

(14, 14)

In [124]:
turkey_companies_df = set_dataframe(turkey_companies_info2)
turkey_companies_df

,symbol,shortName,country,averageVolume,currentPrice,currency,sector,industry,profitMargins,operatingMargins,...,priceToSalesTrailing12Months,priceToBook,enterpriseToRevenue,beta,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiftyDayAverage,twoHundredDayAverage,trailingAnnualDividendYield,fiveYearAvgDividendYield
0,ISBTR.IS,IS BANKASI (B),Turkey,6,30000.000,TRY,Financial Services,Banks—Regional,0.221,0.245,...,17.550,1920.861,3910.672,1.167,6394.402,3487.856,3689.235,4226.907,nan,nan
1,QNBFB.IS,QNB FINANSBANK,Turkey,5273,38.160,TRY,Financial Services,Banks—Regional,0.347,0.421,...,15.399,6.204,19.974,1.604,10.777,4.399,4.598,5.342,nan,nan
2,GARAN.IS,GARANTI BANKASI,Turkey,147291122,9.390,TRY,Financial Services,Banks—Regional,0.308,0.370,...,1.447,0.586,1.050,1.201,1.257,0.749,1.095,0.966,0.016,nan
3,KLNMA.IS,T. KALKINMA BANK.,Turkey,29358,23.660,TRY,Financial Services,Banks—Regional,0.669,0.850,...,35.185,9.513,56.718,-0.236,5.479,2.209,2.357,2.607,nan,nan
4,AKBNK.IS,AKBANK,Turkey,65229093,5.550,TRY,Financial Services,Banks—Regional,0.353,0.473,...,1.358,0.429,2.918,1.044,0.887,0.541,0.654,0.621,0.022,nan
5,ISCTR.IS,IS BANKASI (C),Turkey,40740588,5.270,TRY,Financial Services,Banks—Regional,0.221,0.245,...,0.688,0.337,3.221,1.167,0.857,0.542,0.642,0.611,nan,nan
6,YKBNK.IS,YAPI VE KREDI BANK.,Turkey,193645340,2.450,TRY,Financial Services,Banks—Regional,0.351,0.457,...,1.151,0.395,5.266,1.266,0.391,0.229,0.294,0.270,nan,nan
7,SAHOL.IS,SABANCI HOLDING,Turkey,15962974,9.670,TRY,Financial Services,Banks—Regional,0.241,0.486,...,0.792,0.475,5.811,0.940,1.385,0.856,1.161,1.074,0.036,3.030
8,VAKBN.IS,VAKIFLAR BANKASI,Turkey,37962129,3.440,TRY,Financial Services,Banks—Regional,0.275,0.340,...,0.963,0.281,9.873,1.006,0.581,0.384,0.411,0.416,nan,nan
9,HALKB.IS,T. HALK BANKASI,Turkey,42513503,4.670,TRY,Financial Services,Banks—Regional,0.132,0.125,...,1.140,0.274,8.886,0.877,0.723,0.478,0.550,0.547,nan,nan


In [125]:
turkey_companies_df = turkey_companies_df[turkey_companies_df.isnull().sum(axis=1) < 16]
turkey_companies_df = turkey_companies_df[turkey_companies_df['averageVolume'] > 500000]
len(turkey_companies_df)

10

In [126]:
turkey_companies_df

,symbol,shortName,country,averageVolume,currentPrice,currency,sector,industry,profitMargins,operatingMargins,...,priceToSalesTrailing12Months,priceToBook,enterpriseToRevenue,beta,fiftyTwoWeekHigh,fiftyTwoWeekLow,fiftyDayAverage,twoHundredDayAverage,trailingAnnualDividendYield,fiveYearAvgDividendYield
2,GARAN.IS,GARANTI BANKASI,Turkey,147291122,9.390,TRY,Financial Services,Banks—Regional,0.308,0.370,...,1.447,0.586,1.050,1.201,1.257,0.749,1.095,0.966,0.016,nan
4,AKBNK.IS,AKBANK,Turkey,65229093,5.550,TRY,Financial Services,Banks—Regional,0.353,0.473,...,1.358,0.429,2.918,1.044,0.887,0.541,0.654,0.621,0.022,nan
5,ISCTR.IS,IS BANKASI (C),Turkey,40740588,5.270,TRY,Financial Services,Banks—Regional,0.221,0.245,...,0.688,0.337,3.221,1.167,0.857,0.542,0.642,0.611,nan,nan
6,YKBNK.IS,YAPI VE KREDI BANK.,Turkey,193645340,2.450,TRY,Financial Services,Banks—Regional,0.351,0.457,...,1.151,0.395,5.266,1.266,0.391,0.229,0.294,0.270,nan,nan
7,SAHOL.IS,SABANCI HOLDING,Turkey,15962974,9.670,TRY,Financial Services,Banks—Regional,0.241,0.486,...,0.792,0.475,5.811,0.940,1.385,0.856,1.161,1.074,0.036,3.030
8,VAKBN.IS,VAKIFLAR BANKASI,Turkey,37962129,3.440,TRY,Financial Services,Banks—Regional,0.275,0.340,...,0.963,0.281,9.873,1.006,0.581,0.384,0.411,0.416,nan,nan
9,HALKB.IS,T. HALK BANKASI,Turkey,42513503,4.670,TRY,Financial Services,Banks—Regional,0.132,0.125,...,1.140,0.274,8.886,0.877,0.723,0.478,0.550,0.547,nan,nan
11,TSKB.IS,T.S.K.B.,Turkey,54790443,1.250,TRY,Financial Services,Banks—Regional,0.562,0.679,...,2.760,0.576,35.141,0.861,0.324,0.121,0.153,0.157,0.027,nan
12,ALBRK.IS,ALBARAKA TURK,Turkey,11662026,1.520,TRY,Financial Services,Banks—Regional,0.201,0.244,...,1.118,0.488,-0.004,1.047,0.319,0.167,0.175,0.188,nan,nan
13,SKBNK.IS,SEKERBANK,Turkey,37425039,1.010,TRY,Financial Services,Banks—Regional,0.106,0.102,...,1.192,0.686,0.989,1.109,0.206,0.117,0.121,0.130,nan,nan


In [127]:
save_to_csv(turkey_companies_df, 'turkey.csv')

## India

In [60]:
india_url = 'https://finance.yahoo.com/screener/unsaved/77c80c39-502d-45dc-a03e-fdf15158467d?dependentField=sector&dependentValues=Financial%20Services&offset=0&count=100'
india_results = get_companies(india_url)

In [168]:
india_results = ['HDFCBANK.NS',
 'HDFCBANK.BO',
 'ICICIBANK.NS',
 'SBIN.NS',
 'KOTAKBANK.NS',
 'ICICIBANK.BO',
 'KOTAKBANK.BO',
 'AXISBANK.NS',
 'SBIN.BO',
 'AXISBANK.BO',
 'INDUSINDBK.NS',
 'BANDHANBNK.NS',
 'INDUSINDBK.BO',
 'PNB.NS',
 'IDBI.NS',
 'IOB.NS',
 'IDBI.BO',
 'AUBANK.NS',
 'YESBANK.NS',
 'YESBANK.BO',
 'IDFCFIRSTB.NS',
 'CANBK.NS',
 'PNB.BO',
 'UNIONBANK.NS',
 'AUBANK.BO',
 'BANKINDIA.NS',
 'BANKBARODA.NS',
 'CENTRALBK.NS',
 'FEDERALBNK.NS',
 'INDIANB.NS',
 'UCOBANK.NS',
 'UNIONBANK.BO',
 'IOB.BO',
 'BANKINDIA.BO',
 'CANBK.BO',
 'MAHABANK.NS',
 'UCOBANK.BO',
 'BANKBARODA.BO',
 'CUB.NS',
 'FEDERALBNK.BO',
 'CUB.BO',
 'RBLBANK.NS',
 'MAHABANK.BO',
 'RBLBANK.BO',
 'PSB.NS',
 'INDIANB.BO',
 'EQUITASBNK.NS',
 'CENTRALBK.BO',
 'CSBBANK.NS',
 'KARURVYSYA.NS',
 'UJJIVANSFB.NS',
 'DCBBANK.NS',
 'J&KBANK.NS',
 'KARURVYSYA.BO',
 'DCBBANK.BO',
 'KTKBANK.NS',
 'SOUTHBANK.NS',
 'SURYODAY.NS',
 'KTKBANK.BO',
 'SOUTHBANK.BO',
 'J&KBANK.BO',
 'PSB.BO',
 'DHANBANK.NS',
 'DHANBANK.BO']

In [169]:
c = CurrencyRates()
india_rate = c.get_rate('INR', 'USD')

In [170]:
india_companies_info = get_companies_info(india_results, 'INR', india_rate, 'India')

HDFCBANK.NS HDFCBANK.BO ICICIBANK.NS SBIN.NS KOTAKBANK.NS ICICIBANK.BO KOTAKBANK.BO AXISBANK.NS SBIN.BO AXISBANK.BO INDUSINDBK.NS BANDHANBNK.NS INDUSINDBK.BO PNB.NS IDBI.NS IOB.NS IDBI.BO AUBANK.NS YESBANK.NS YESBANK.BO IDFCFIRSTB.NS CANBK.NS PNB.BO UNIONBANK.NS AUBANK.BO BANKINDIA.NS BANKBARODA.NS CENTRALBK.NS FEDERALBNK.NS INDIANB.NS UCOBANK.NS UNIONBANK.BO IOB.BO BANKINDIA.BO CANBK.BO MAHABANK.NS UCOBANK.BO BANKBARODA.BO CUB.NS FEDERALBNK.BO CUB.BO RBLBANK.NS MAHABANK.BO RBLBANK.BO PSB.NS INDIANB.BO EQUITASBNK.NS CENTRALBK.BO CSBBANK.NS KARURVYSYA.NS UJJIVANSFB.NS DCBBANK.NS J&KBANK.NS KARURVYSYA.BO DCBBANK.BO KTKBANK.NS SOUTHBANK.NS SURYODAY.NS KTKBANK.BO SOUTHBANK.BO J&KBANK.BO PSB.BO DHANBANK.NS DHANBANK.BO 

In [171]:
india_companies_info2 = filter_info('INR', 'India', india_companies_info.values())
len(india_companies_info), len(india_companies_info2)

(64, 64)

In [71]:
india_companies_df = set_dataframe(india_companies_info2)

# filter too many empty valued rows
india_companies_df = india_companies_df[india_companies_df.isnull().sum(axis=1) < 16]
len(india_companies_df)

36

In [72]:
save_to_csv(india_companies_df, 'india.csv')

## Brazil

In [7]:
brazil_url = 'https://finance.yahoo.com/screener/unsaved/4b52d171-1f9b-42e3-b227-7adbd5dcb46d?dependentField=sector&dependentValues=Financial%20Services&offset=0&count=100'
brazil_results = get_companies(brazil_url)

In [9]:
brazil_results = ['H1SB34.SA',
 'WFCO34.SA',
 'JPMC34.SA',
 'BOAC34.SA',
 'USBC34.SA',
 'PNCS34.SA',
 'UBSG34.SA',
 'ITUB4.SA',
 'ITUB3.SA',
 'INGG34.SA',
 'B1CS34.SA',
 'S1IV34.SA',
 'BBDC4.SA',
 'BBDC3.SA',
 'CTGP34.SA',
 'FFTD34.SA',
 'SANB11.SA',
 'SANB4.SA',
 'SANB3.SA',
 'K1EY34.SA',
 'R1FC34.SA',
 'C1FG34.SA',
 'M1TB34.SA',
 'ITSA4.SA',
 'ITSA3.SA',
 'SBNY34.SA',
 'BBAS3.SA',
 'C1MA34.SA',
 'B1SA34.SA',
 'BIDI3.SA',
 'BIDI4.SA',
 'VLYB34.SA',
 'BPAN4.SA',
 'BSLI4.SA',
 'BSLI3.SA',
 'BRSR3.SA',
 'BRSR6.SA',
 'BRSR5.SA',
 'ABCB4.SA',
 'BMGB4.SA',
 'BEES3.SA',
 'BEES4.SA',
 'BAZA3.SA',
 'BMEB3.SA',
 'BMEB4.SA',
 'BRIV4.SA',
 'BRIV3.SA',
 'BGIP3.SA',
 'BGIP4.SA',
 'MERC3.SA',
 'PINE4.SA',
 'BMIN4.SA',
 'HGFF11.SA',
 'DBAG34.SA',
 'HGPO11.SA',
 'H1DB34.SA',
 'HGRU11.SA',
 'MODL11.SA',
 'BIDI11.SA',
 'BCSA34.SA',
 'S1MF34.SA',
 'BILB34.SA',
 'I1BN34.SA',
 'K1BF34.SA',
 'C1SU34.SA',
 'M1UF34.SA',
 'CRFF11.SA']

In [10]:
c = CurrencyRates()
brazil_rate = c.get_rate('BRL', 'USD')

In [11]:
brazil_companies_info = get_companies_info(brazil_results, 'BRL', brazil_rate, 'Brazil')

H1SB34.SA WFCO34.SA JPMC34.SA BOAC34.SA USBC34.SA PNCS34.SA UBSG34.SA ITUB4.SA ITUB3.SA INGG34.SA B1CS34.SA S1IV34.SA BBDC4.SA BBDC3.SA CTGP34.SA FFTD34.SA SANB11.SA SANB4.SA SANB3.SA K1EY34.SA R1FC34.SA C1FG34.SA M1TB34.SA ITSA4.SA ITSA3.SA SBNY34.SA BBAS3.SA C1MA34.SA B1SA34.SA BIDI3.SA BIDI4.SA VLYB34.SA BPAN4.SA BSLI4.SA BSLI3.SA BRSR3.SA BRSR6.SA BRSR5.SA ABCB4.SA BMGB4.SA BEES3.SA BEES4.SA BAZA3.SA BMEB3.SA BMEB4.SA BRIV4.SA BRIV3.SA BGIP3.SA BGIP4.SA MERC3.SA PINE4.SA BMIN4.SA HGFF11.SA DBAG34.SA HGPO11.SA H1DB34.SA HGRU11.SA MODL11.SA BIDI11.SA BCSA34.SA S1MF34.SA BILB34.SA I1BN34.SA K1BF34.SA C1SU34.SA M1UF34.SA CRFF11.SA 

In [12]:
brazil_companies_info2 = filter_info('BRL', 'Brazil', brazil_companies_info.values())
len(brazil_companies_info), len(brazil_companies_info2)

(67, 48)

In [13]:
brazil_companies_df = set_dataframe(brazil_companies_info2)

In [135]:
brazil_companies_df = brazil_companies_df[brazil_companies_df.isnull().sum(axis=1) < 16]
brazil_companies_df = brazil_companies_df[brazil_companies_df['averageVolume'] > 50000]
brazil_companies_df = brazil_companies_df[brazil_companies_df['country'] != '']
len(brazil_companies_df)

17

In [136]:
save_to_csv(brazil_companies_df, 'brazil.csv')

## Indonesia

In [16]:
indonesia_url = 'https://finance.yahoo.com/screener/unsaved/2e08226c-af25-46f3-a72e-83905c7a467e?dependentField=sector&dependentValues=Financial%20Services&offset=0&count=100'
indonesia_results = get_companies(indonesia_url)

In [20]:
indonesia_results = ['BBCA.JK',
 'BBRI.JK',
 'BMRI.JK',
 'ARTO.JK',
 'BBNI.JK',
 'BRIS.JK',
 'BNLI.JK',
 'MEGA.JK',
 'AGRO.JK',
 'BANK.JK',
 'BBHI.JK',
 'BDMN.JK',
 'BNII.JK',
 'BTPS.JK',
 'BINA.JK',
 'BNGA.JK',
 'BTPN.JK',
 'BSIM.JK',
 'BBSI.JK',
 'PNBN.JK',
 'NISP.JK',
 'BBTN.JK',
 'BBKP.JK',
 'BJBR.JK',
 'BBYB.JK',
 'BJTM.JK',
 'MAYA.JK',
 'BABP.JK',
 'BBMD.JK',
 'BMAS.JK',
 'MCOR.JK',
 'SDRA.JK',
 'NOBU.JK',
 'BKSW.JK',
 'MASB.JK',
 'PNBS.JK',
 'AGRS.JK',
 'DNAR.JK',
 'BNBA.JK',
 'BACA.JK',
 'BGTG.JK',
 'INPC.JK',
 'AMAR.JK',
 'BVIC.JK',
 'BCIC.JK',
 'BEKS.JK',
 'MASB-W.JK',
 'BANK-W.JK']

In [30]:
c = CurrencyRates()
indonesia_rate = c.get_rate('IDR', 'USD')
# indonesia_rate = 0.000070

In [31]:
indonesia_companies_info = get_companies_info(indonesia_results, 'Rp', indonesia_rate, 'Indonesia')

BBCA.JK BBRI.JK BMRI.JK ARTO.JK BBNI.JK BRIS.JK BNLI.JK MEGA.JK AGRO.JK BANK.JK BBHI.JK BDMN.JK BNII.JK BTPS.JK BINA.JK BNGA.JK BTPN.JK BSIM.JK BBSI.JK PNBN.JK NISP.JK BBTN.JK BBKP.JK BJBR.JK BBYB.JK BJTM.JK MAYA.JK BABP.JK BBMD.JK BMAS.JK MCOR.JK SDRA.JK NOBU.JK BKSW.JK MASB.JK PNBS.JK AGRS.JK DNAR.JK BNBA.JK BACA.JK BGTG.JK INPC.JK AMAR.JK BVIC.JK BCIC.JK BEKS.JK MASB-W.JK BANK-W.JK 

In [36]:
indonesia_companies_info2 = filter_info('IDR', 'Indonesia', indonesia_companies_info.values())
len(indonesia_companies_info), len(indonesia_companies_info2)

(48, 48)

In [37]:
indonesia_companies_df = set_dataframe(indonesia_companies_info2)

In [139]:
indonesia_companies_df = indonesia_companies_df[indonesia_companies_df.isnull().sum(axis=1) < 16]
indonesia_companies_df = indonesia_companies_df[indonesia_companies_df['averageVolume'] > 50000]
len(indonesia_companies_df)

43

In [140]:
save_to_csv(indonesia_companies_df, 'indonesia.csv')

## Thailand

In [50]:
thailand_url = 'https://finance.yahoo.com/screener/unsaved/c849b845-243c-485d-9295-05d34ee18eb5?dependentField=sector&dependentValues=Financial%20Services'
thailand_results = get_companies(thailand_url)

In [52]:
thailand_results = ['SCB-R.BK',
 'KBANK-R.BK',
 'BAY-R.BK',
 'SCB.BK',
 'KBANK.BK',
 'BAY.BK',
 'BBL-R.BK',
 'BBL.BK',
 'KTB-R.BK',
 'KTB.BK',
 'KKP-R.BK',
 'TTB-R.BK',
 'TTB.BK',
 'TISCO-R.BK',
 'TISCO.BK',
 'KKP.BK',
 'CIMBT-R.BK',
 'CIMBT.BK',
 'LHFG-R.BK',
 'LHFG.BK']

In [53]:
c = CurrencyRates()
thailand_rate = c.get_rate('THB', 'USD')
# indonesia_rate = 0.0013

In [54]:
thailand_companies_info = get_companies_info(thailand_results, 'THB', thailand_rate, 'Thailand')

SCB-R.BK KBANK-R.BK BAY-R.BK SCB.BK KBANK.BK BAY.BK BBL-R.BK BBL.BK KTB-R.BK KTB.BK KKP-R.BK TTB-R.BK TTB.BK TISCO-R.BK TISCO.BK KKP.BK CIMBT-R.BK CIMBT.BK LHFG-R.BK LHFG.BK 

In [55]:
thailand_companies_info2 = filter_info('THB', 'Thailand', thailand_companies_info.values())
len(thailand_companies_info), len(thailand_companies_info2)

(20, 20)

In [145]:
thailand_companies_df = set_dataframe(thailand_companies_info2)

In [141]:
thailand_companies_df = thailand_companies_df[thailand_companies_df.isnull().sum(axis=1) < 16]
thailand_companies_df = thailand_companies_df[thailand_companies_df['averageVolume'] > 50000]
len(thailand_companies_df)

13

In [142]:
save_to_csv(thailand_companies_df, 'thailand.csv')

## Malaysia

In [18]:
malaysia_url = 'https://finance.yahoo.com/screener/unsaved/1187ace3-5d9e-43e8-9345-782aea5cd1df?dependentField=sector&dependentValues=Financial%20Services'
malaysia_results = get_companies(malaysia_url)

In [41]:
malaysia_results = ['5819.KL',
 '5258.KL',
 '1066.KL',
 '1171.KL',
 '1082.KL',
 '1015.KL',
 '1295.KL',
 '5185.KL',
 '1155.KL',
 '1023.KL',
 '2488.KL']

In [42]:
c = CurrencyRates()
malaysia_rate = c.get_rate('MYR', 'USD')
# indonesia_rate = 0.0013

In [43]:
malaysia_companies_info = get_companies_info(malaysia_results, 'MYR', malaysia_rate, 'Malaysia')

5819.KL 5258.KL 1066.KL 1171.KL 1082.KL 1015.KL 1295.KL 5185.KL 1155.KL 1023.KL 2488.KL 

In [44]:
malaysia_companies_info2 = filter_info('MYR', 'Malaysia', malaysia_companies_info.values())
len(malaysia_companies_info), len(malaysia_companies_info2)

(11, 11)

In [148]:
malaysia_companies_df = set_dataframe(malaysia_companies_info2)
len(malaysia_companies_df)

11

In [149]:
malaysia_companies_df = malaysia_companies_df[malaysia_companies_df.isnull().sum(axis=1) < 16]
malaysia_companies_df = malaysia_companies_df[malaysia_companies_df['averageVolume'] > 50000]
len(malaysia_companies_df)

11

In [150]:
save_to_csv(malaysia_companies_df, 'malaysia.csv')

## Argentina

In [62]:
argentina_url = 'https://finance.yahoo.com/screener/unsaved/bb435016-a635-4261-af7f-845e34c32b5d?dependentField=sector&dependentValues=Financial%20Services'
argentina_results = get_companies(argentina_url)

In [65]:
argentina_results = ['C.BA',
 'USB.BA',
 'JPM.BA',
 'BCS.BA',
 'HSBC.BA',
 'WFC.BA',
 'WBK.BA',
 'BBV.BA',
 'BBD.BA',
 'ING.BA',
 'CS.BA',
 'LYG.BA',
 'BSBR.BA',
 'GGAL.BA',
 'BMA.BA',
 'BBAR.BA',
 'BRIO.BA',
 'BRIO6.BA',
 'BPAT.BA',
 'SUPV.BA',
 'BHIP.BA',
 'ITUB.BA']

In [67]:
c = CurrencyRates()
# argentina_rate = c.get_rate('ARS', 'USD')

argentina_rate = 0.010

In [79]:
argentina_companies_info = get_companies_info(argentina_results, 'ARS', argentina_rate, 'Argentina')

C.BA USB.BA JPM.BA BCS.BA HSBC.BA WFC.BA WBK.BA BBV.BA BBD.BA ING.BA CS.BA LYG.BA BSBR.BA GGAL.BA BMA.BA BBAR.BA BRIO.BA BRIO6.BA BPAT.BA SUPV.BA BHIP.BA ITUB.BA 

In [80]:
argentina_companies_info2 = filter_info('ARS', 'Argentina', argentina_companies_info.values())
len(argentina_companies_info), len(argentina_companies_info2)

(22, 8)

In [154]:
argentina_companies_df = set_dataframe(argentina_companies_info2)

In [155]:
argentina_companies_df = argentina_companies_df[argentina_companies_df.isnull().sum(axis=1) < 16]
argentina_companies_df = argentina_companies_df[argentina_companies_df['averageVolume'] > 50000]
len(argentina_companies_df)

4

In [156]:
save_to_csv(argentina_companies_df, 'argentina.csv')

## Russia

In [94]:
russia_url = 'https://finance.yahoo.com/screener/unsaved/38fbbd9b-954a-428a-8b96-8e2152b10264?dependentField=sector&dependentValues=Financial%20Services'
russia_results = get_companies(russia_url)

In [96]:
russia_results = ['SBER.ME',
 'SBERP.ME',
 'TCSG.ME',
 'VTBR.ME',
 'CBOM.ME',
 'ROSB.ME',
 'AVAN.ME',
 'BSPB.ME',
 'USBN.ME',
 'PRMB.ME',
 'ALBK.ME',
 'RDRB.ME',
 'KUZB.ME',
 'BSPBP.ME']

In [97]:
c = CurrencyRates()
russia_rate = c.get_rate('RUB', 'USD')

In [98]:
russia_companies_info = get_companies_info(russia_results, 'RUB', russia_rate, 'Russia')

SBER.ME SBERP.ME TCSG.ME VTBR.ME CBOM.ME ROSB.ME AVAN.ME BSPB.ME USBN.ME PRMB.ME ALBK.ME RDRB.ME KUZB.ME BSPBP.ME 

In [99]:
russia_companies_info2 = filter_info('RUB', 'Russia', russia_companies_info.values())
len(russia_companies_info), len(russia_companies_info2)

(14, 13)

In [164]:
russia_companies_df = set_dataframe(russia_companies_info2)

In [165]:
russia_companies_df = russia_companies_df[russia_companies_df.isnull().sum(axis=1) < 16]
russia_companies_df = russia_companies_df[russia_companies_df['averageVolume'] > 50000]
russia_companies_df = russia_companies_df[russia_companies_df.symbol != 'SBERP.ME']
len(russia_companies_df)

6

In [166]:
save_to_csv(russia_companies_df, 'russia.csv')

# Save all dataset to a csv

In [1]:
tur = pd.read_csv('turkey.csv')
ind = pd.read_csv('india.csv')
brz = pd.read_csv('brazil.csv')
idr = pd.read_csv('indonesia.csv')
thb = pd.read_csv('thailand.csv')
mry = pd.read_csv('malaysia.csv')
ars = pd.read_csv('argentina.csv')
rub = pd.read_csv('russia.csv')

NameError: name 'pd' is not defined

In [179]:
# banks_tickers = turkey_results+india_results+brazil_results+indonesia_results+thailand_results+malaysia_results+argentina_results+russia_results
# banks_info = list(turkey_companies_info)+list(india_companies_info)+list(brazil_companies_info)+list(indonesia_companies_info)+list(thailand_companies_info)+list(malaysia_companies_info)+list(argentina_companies_info)+list(russia_companies_info)
# banks_filtered = turkey_companies_info+india_companies_info+brazil_companies_info+indonesia_companies_info+thailand_companies_info+malaysia_companies_info+argentina_companies_info+russia_companies_info


all_banks_developing_df = pd.concat([tur,ind,brz,idr,thb,mry,ars,rub], ignore_index=True)
all_banks_developing_df.to_csv('all_banks_developing.csv')